In [1]:
%pip install transformers torch datasets icecream accelerate trl --no-cache-dir -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 23.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 45.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 KB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 KB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 KB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 KB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 KB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 KB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 51.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 44.7 MB/s eta 0:00:0000:0100

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from icecream import ic

# Carregar o modelo e o tokenizador GPT-2
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Definir um token de preenchimento
tokenizer.pad_token = tokenizer.eos_token

ic.configureOutput(prefix="Model confgi")
ic(model)

# Carregar o dataset de diálogos
#dataset = load_dataset("Osondu/reddit_autism_dataset", trust_remote_code=True )
#dataset_autism = load_dataset("Osondu/reddit_autism_dataset", split="train" )
# Verificar a estrutura do dataset


/home/techgamespi/Área de Trabalho/ia/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Model confgimodel: GPT2LMHeadModel(
                     (transformer): GPT2Model(
                       (wte): Embedding(50257, 768)
                       (wpe): Embedding(1024, 768)
                       (drop): Dropout(p=0.1, inplace=False)
                       (h): ModuleList(
                         (0-11): 12 x GPT2Block(
                           (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                           (attn): GPT2Attention(
                             (c_attn): Conv1D()
                             (c_proj): Conv1D()
                             (attn_dropout): Dropout(p=0.1, inplace=False)
                             (resid_dropout): Dropout(p=0.1, inplace=False)
  

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Preparando dados de treino

In [4]:
def gen_batches_train():
    dataset = load_dataset("Osondu/reddit_autism_dataset",  streaming=True,  split="train")
    ic.configureOutput(prefix="Dataset autism\n")
    ic(dataset)

    total_samples = 800
    val_pct = 0.1
    train_limit = int(total_samples * (1 - val_pct))
    counter = 0

    for row in dataset:
        if counter >= train_limit:
            break

        inst = row["instructions"]
        input = row["input"]
        output = row["output"]

        chat_template = f"<s>[INST] {inst}\nInput: {input} [/INST] {output}</s>"
        tokenized_output = tokenizer(chat_template)
        yield {'text': chat_template}
        counter += 1


def gen_batches_val():
    dataset = load_dataset("Osondu/reddit_autism_dataset",  streaming=True,  split="train")
    total_samples = 800
    val_pct = 0.1
    train_limit = int(total_samples * (1 - val_pct))
    counter = 0

    for row in dataset:
        if counter >= total_samples:
           break

        inst = row["instructions"]
        input = row["input"]
        output = row["output"]

        chat_template = f"<s>[INST] {inst}\nInput: {input} [/INST] {output}</s>"
        tokenized_output = tokenizer(chat_template)
        yield {'text': chat_template}
        counter += 1

### Criando os datasets de treino e valdiação

In [5]:
train_gen = Dataset.from_generator(gen_batches_train)
val_gen = Dataset.from_generator(gen_batches_val)

Generating train split: 0 examples [00:00, ? examples/s]Dataset autism
dataset: IterableDataset({
                            features: ['instructions', 'input', 'output'],
                            n_shards: 1
                        })
Token indices sequence length is longer than the specified maximum sequence length for this model (5393 > 1024). Running this sequence through the model will result in indexing errors
Generating train split: 720 examples [00:23, 30.56 examples/s]
Generating train split: 800 examples [00:21, 36.99 examples/s] 


In [6]:
# Ajustar a função de tokenização de acordo com os campos do dataset
def tokenize_function(examples):
    # "utterance" é um dicionário contendo listas de strings em "text"
    ic.configureOutput(prefix="Tokenize function\n")
    ic(examples)
    texts = examples["text"]
     #Concatenar todas as linhas de diálogo em um único texto
    concatenated_texts = [" ".join(text_list) for text_list in texts]
    return tokenizer(concatenated_texts, padding="max_length", truncation=True, max_length=512)

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  
    optim="adamw_torch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)
# Treinar o modelo
trainer = SFTTrainer(
    model=model,
    dataset_text_field="text",
    args=training_args,
    train_dataset=train_gen,
    eval_dataset=val_gen,
    tokenizer=tokenizer,
)

trainer.train()

# Salvar o modelo treinado
model.save_pretrained("../chatbot_model")
tokenizer.save_pretrained("../chatbot_model")

/home/techgamespi/Área de Trabalho/ia/env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/techgamespi/Área de Trabalho/ia/env/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/techgamespi/Área de Trabalho/ia/env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/techgamespi/Área de Trabalho/ia/env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `datase

{'eval_loss': 2.864370107650757, 'eval_runtime': 1351.9429, 'eval_samples_per_second': 0.592, 'eval_steps_per_second': 0.296, 'epoch': 1.0}


 46%|████▋     | 500/1080 [2:01:58<1:57:00, 12.10s/it]  

{'loss': 3.0526, 'grad_norm': 6.836942195892334, 'learning_rate': 1.0740740740740742e-05, 'epoch': 1.39}


 67%|██████▋   | 720/1080 [3:07:43<1:07:04, 11.18s/it]

...

 93%|█████████▎| 1000/1080 [4:02:21<15:43, 11.79s/it]   

{'loss': 2.944, 'grad_norm': 6.776059627532959, 'learning_rate': 1.4814814814814815e-06, 'epoch': 2.78}


100%|██████████| 1080/1080 [4:40:48<00:00, 15.60s/it]


{'eval_loss': 2.799729108810425, 'eval_runtime': 1359.1074, 'eval_samples_per_second': 0.589, 'eval_steps_per_second': 0.294, 'epoch': 3.0}
{'train_runtime': 16848.89, 'train_samples_per_second': 0.128, 'train_steps_per_second': 0.064, 'train_loss': 2.986323278921622, 'epoch': 3.0}


('../chatbot_model/tokenizer_config.json',
 '../chatbot_model/special_tokens_map.json',
 '../chatbot_model/vocab.json',
 '../chatbot_model/merges.txt',
 '../chatbot_model/added_tokens.json')